## How to build TensorFlow models with the Keras functional API: examples, code, and notebooks

Click the image below to read the post online.

<a target="_blank" href="https://www.machinelearningnuggets.com/tensorflow-keras-functional-api
"><img src="https://www.machinelearningnuggets.com/ezoimgfmt/digitalpress.fra1.cdn.digitaloceanspaces.com/mhujhsj/2022/07/logho-1.png?ezimgfmt=ng:webp/ngcb1" alt="Open in ML Nuggets"></a>

In [ ]:
import tensorflow
tensorflow.__version__

In [ ]:
from tensorflow import keras 
from tensorflow.keras import layers
parameters = {"shape":28, "activation": "relu", "classes": 10, "units":12, "optimizer":"adam", "epochs":1,"kernel_size":3,"pool_size":2, "dropout":0.5}
# Setup the layers
model = keras.Sequential(
  [
      layers.Conv2D(32, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), input_shape =(parameters["shape"], parameters["shape"], 1),activation=parameters["activation"]),
      layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"])),
      layers.Conv2D(64, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), activation=parameters["activation"]),
      layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"])),
      layers.Flatten(),
      layers.Dropout(parameters["dropout"]),
      layers.Dense(parameters["classes"], activation="softmax"),
  ]
)

In [ ]:
model.summary()

In [ ]:
keras.utils.plot_model(model, "model.png")


In [ ]:

keras.utils.plot_model(model, "model.png",show_shapes=True)


In [ ]:
  inputs = keras.Input(shape=(parameters["shape"], parameters["shape"], 1))


In [ ]:
inputs.shape

In [ ]:
inputs.dtype


In [ ]:
conv2D = layers.Conv2D(32, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), input_shape =(parameters["shape"], parameters["shape"], 1),activation=parameters["activation"])
x = conv2D(inputs)

In [ ]:
parameters = {"shape":28, "activation": "relu", "classes": 10, "units":12, "optimizer":"adam", "epochs":1,"kernel_size":3,"pool_size":2, "dropout":0.5}
inputs = keras.Input(shape=(parameters["shape"], parameters["shape"], 1))
conv2D = layers.Conv2D(32, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), input_shape =(parameters["shape"], parameters["shape"], 1),activation=parameters["activation"])(inputs)
maxPooling2D = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D)
conv2D_2 =layers.Conv2D(64, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), activation=parameters["activation"])(maxPooling2D)
maxPooling2D_2 = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D_2)
flatten =   layers.Flatten()(maxPooling2D_2)
dropout = layers.Dropout(parameters["dropout"])(flatten)
outputs = layers.Dense(parameters["classes"], activation="softmax")(dropout)


In [ ]:
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")


In [ ]:
keras.utils.plot_model(model, "model.png",show_shapes=True)


## Training and evaluation of Functional API models

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()

x_train = x_train.astype("float32") / 255
x_test = x_test.astype("float32") / 255

model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.RMSprop(),
    metrics=["accuracy"],
)

history = model.fit(x_train, y_train, batch_size=64, epochs=2, validation_split=0.2)

test_scores = model.evaluate(x_test, y_test, verbose=2)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
model.save("saved_model")
del model
model = keras.models.load_model("saved_model")
model.summary()

In [ ]:
# model.summary(expand_nested=True)

## How to convert a Functional model to a Sequential API model

In [ ]:
seq_model = keras.models.Sequential()
for layer in model.layers:
    seq_model.add(layer)
seq_model.summary()

## Convert Sequential model to Functional model

In [ ]:
inputs = keras.Input(batch_shape=seq_model.layers[0].input_shape)
x = inputs
for layer in seq_model.layers:
    x = layer(x) 
outputs = x
func_model = keras.Model(inputs=inputs, outputs=outputs, name="func_mnist_model")

In [ ]:
func_model.summary()

## Standard network models

### Multilayer perceptron

In [ ]:
inputs = keras.Input(shape=(parameters["shape"], parameters["shape"], 1))
dense1 = layers.Dense(128)(inputs)
dropout = layers.Dropout(parameters["dropout"])(dense1)
dense2 = layers.Dense(128)(dropout)
dropout1 = layers.Dropout(parameters["dropout"])(dense2)
outputs = layers.Dense(parameters["classes"], activation="softmax")(dropout1)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
keras.utils.plot_model(model, "model.png",show_shapes=True)


### Convolutional Neural Network

In [ ]:
inputs = keras.Input(shape=(parameters["shape"], parameters["shape"], 1))
conv2D = layers.Conv2D(32, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), input_shape =(parameters["shape"], parameters["shape"], 1),activation=parameters["activation"])(inputs)
maxPooling2D = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D)
conv2D_2 =layers.Conv2D(64, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), activation=parameters["activation"])(maxPooling2D)
maxPooling2D_2 = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D_2)
flatten =   layers.Flatten()(maxPooling2D_2)
dropout = layers.Dropout(parameters["dropout"])(flatten)
outputs = layers.Dense(parameters["classes"], activation="softmax")(dropout)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
keras.utils.plot_model(model, "model.png",show_shapes=True)



### Recurrent Neural Network

In [ ]:
inputs = keras.Input(784,)
embedding = layers.Embedding(512, 64, input_length=1024)(inputs)
bidirectional1 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embedding)
bidirectional2 = layers.Bidirectional(layers.LSTM(64,))(bidirectional1)
dense1 = layers.Dense(32, activation='relu')(bidirectional2)
outputs = layers.Dense(1, activation='sigmoid')(dense1)
model = keras.Model(inputs=inputs, outputs=outputs, name="lstm_model")
keras.utils.plot_model(model, "model.png",show_shapes=True)

## Shared input layer

In [ ]:
inputs = keras.Input(shape=(parameters["shape"], parameters["shape"], 1))
conv2D = layers.Conv2D(32, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), input_shape =(parameters["shape"], parameters["shape"], 1),activation=parameters["activation"])(inputs)
maxPooling2D = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D)
flatten1 =   layers.Flatten()(maxPooling2D)

conv2D_2 = layers.Conv2D(64, kernel_size=(parameters["kernel_size"], parameters["kernel_size"]), activation=parameters["activation"])(inputs)
maxPooling2D_2 = layers.MaxPooling2D(pool_size=(parameters["pool_size"], parameters["pool_size"]))(conv2D_2)
flatten2 =   layers.Flatten()(maxPooling2D_2)

# merge layers
merged_layers = layers.concatenate([flatten1, flatten2])

dropout = layers.Dropout(parameters["dropout"])(merged_layers)
outputs = layers.Dense(parameters["classes"], activation="softmax")(dropout)
model = keras.Model(inputs=inputs, outputs=outputs, name="mnist_model")
keras.utils.plot_model(model, "model.png",show_shapes=True)

## Shared feature extraction layer


In [ ]:
inputs = keras.Input(784,)
embedding = layers.Embedding(512, 64, input_length=1024)(inputs)
bidirectional1 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embedding)
bidirectional2 = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(embedding)

# merge layers
merged_layers = layers.concatenate([bidirectional1, bidirectional2])

dense1 = layers.Dense(32, activation='relu')(merged_layers)
outputs = layers.Dense(1, activation='sigmoid')(dense1)
model = keras.Model(inputs=inputs, outputs=outputs, name="lstm_model")
keras.utils.plot_model(model, "model.png",show_shapes=True)

## Multiple input model

In [ ]:
input1 = keras.Input(shape=(16,))
x1 =layers.Dense(8, activation='relu')(input1)
input2 = layers.Input(shape=(32,))
x2 = layers.Dense(8, activation='relu')(input2)
# equivalent to `added = tf.keras.layers.add([x1, x2])`
added = layers.Add()([x1, x2])
out = layers.Dense(4)(added)
model = keras.Model(inputs=[input1, input2], outputs=out)
keras.utils.plot_model(model, "model.png",show_shapes=True)

### Multiple output model

In [ ]:
image_input = keras.Input(shape=(parameters["shape"], parameters["shape"], 3), name="images") 
x = layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')(image_input)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Conv2D(filters=32,kernel_size=(3,3), activation='relu')(x)
x = layers.Dropout(0.25)(x)
x = layers.Conv2D(filters=64,kernel_size=(3,3), activation='relu')(x)
x = layers.MaxPooling2D(pool_size=(2,2))(x)

x = layers.Dropout(0.25)(x)
x = layers.Flatten()(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.25)(x)

gender_prediction = layers.Dense(3, activation='softmax')(x)
age_prediction = layers.Dense(3, activation='softmax')(x)

model = keras.Model(
    inputs=image_input,
    outputs=[gender_prediction, age_prediction],
)
keras.utils.plot_model(model, "model.png",show_shapes=True)

## Use the same graph of layers to define multiple models

In [ ]:
encoder_input = keras.Input(shape=(28, 28, 1), name="img")
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
# encoder.summary()

x = layers.Reshape((4, 4, 1))(encoder_output)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation="relu")(x)

autoencoder = keras.Model(encoder_input, decoder_output, name="autoencoder")
# autoencoder.summary()
keras.utils.plot_model(autoencoder, "autoencoder.png",show_shapes=True)

In [ ]:
keras.utils.plot_model(autoencoder, "autoencoder.png",show_shapes=True)


## All models are callable, just like layers

In [ ]:
encoder_input = keras.Input(shape=(28, 28, 1), name="original_img")
x = layers.Conv2D(16, 3, activation="relu")(encoder_input)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.MaxPooling2D(3)(x)
x = layers.Conv2D(32, 3, activation="relu")(x)
x = layers.Conv2D(16, 3, activation="relu")(x)
encoder_output = layers.GlobalMaxPooling2D()(x)

encoder = keras.Model(encoder_input, encoder_output, name="encoder")
encoder.summary()

decoder_input = keras.Input(shape=(16,), name="encoded_img")
x = layers.Reshape((4, 4, 1))(decoder_input)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
x = layers.Conv2DTranspose(32, 3, activation="relu")(x)
x = layers.UpSampling2D(3)(x)
x = layers.Conv2DTranspose(16, 3, activation="relu")(x)
decoder_output = layers.Conv2DTranspose(1, 3, activation="relu")(x)

decoder = keras.Model(decoder_input, decoder_output, name="decoder")
decoder.summary()

autoencoder_input = keras.Input(shape=(28, 28, 1), name="img")
encoded_img = encoder(autoencoder_input)
decoded_img = decoder(encoded_img)
autoencoder = keras.Model(autoencoder_input, decoded_img, name="autoencoder")
autoencoder.summary()

## How to ensemble models

In [ ]:
def get_model():
    inputs = keras.Input(shape=(128,))
    outputs = layers.Dense(1)(inputs)
    return keras.Model(inputs, outputs)


model1 = get_model()
model2 = get_model()
model3 = get_model()

inputs = keras.Input(shape=(128,))
y1 = model1(inputs)
y2 = model2(inputs)
y3 = model3(inputs)
outputs = layers.average([y1, y2, y3])
ensemble_model = keras.Model(inputs=inputs, outputs=outputs)

## Models with multiple inputs and outputs

In [ ]:
num_tags = 12  # Number of unique issue tags
num_words = 10000  # Size of vocabulary obtained when preprocessing text data
num_departments = 4  # Number of departments for predictions

title_input = keras.Input(
    shape=(None,), name="title"
)  # Variable-length sequence of ints
body_input = keras.Input(shape=(None,), name="body")  # Variable-length sequence of ints
tags_input = keras.Input(
    shape=(num_tags,), name="tags"
)  # Binary vectors of size `num_tags`

# Embed each word in the title into a 64-dimensional vector
title_features = layers.Embedding(num_words, 64)(title_input)
# Embed each word in the text into a 64-dimensional vector
body_features = layers.Embedding(num_words, 64)(body_input)

# Reduce sequence of embedded words in the title into a single 128-dimensional vector
title_features = layers.LSTM(128)(title_features)
# Reduce sequence of embedded words in the body into a single 32-dimensional vector
body_features = layers.LSTM(32)(body_features)

# Merge all available features into a single large vector via concatenation
x = layers.concatenate([title_features, body_features, tags_input])

# Stick a logistic regression for priority prediction on top of the features
priority_pred = layers.Dense(1, name="priority")(x)
# Stick a department classifier on top of the features
department_pred = layers.Dense(num_departments, name="department")(x)

# Instantiate an end-to-end model predicting both priority and department
model = keras.Model(
    inputs=[title_input, body_input, tags_input],
    outputs=[priority_pred, department_pred],
)

In [ ]:
keras.utils.plot_model(model, "multi_input_and_output_model.png", show_shapes=True)


## Compile model with multiple inputs

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=[
        keras.losses.BinaryCrossentropy(from_logits=True),
        keras.losses.CategoricalCrossentropy(from_logits=True),
    ],
    loss_weights=[1.0, 0.2],
)

In [ ]:
model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss={
        "priority": keras.losses.BinaryCrossentropy(from_logits=True),
        "department": keras.losses.CategoricalCrossentropy(from_logits=True),
    },
    loss_weights={"priority": 1.0, "department": 0.2},
)

## Training models with multiple inputs

In [ ]:
# Dummy input data
import numpy as np
title_data = np.random.randint(num_words, size=(1280, 10))
body_data = np.random.randint(num_words, size=(1280, 100))
tags_data = np.random.randint(2, size=(1280, num_tags)).astype("float32")

# Dummy target data
priority_targets = np.random.random(size=(1280, 1))
dept_targets = np.random.randint(2, size=(1280, num_departments))

model.fit(
    {"title": title_data, "body": body_data, "tags": tags_data},
    {"priority": priority_targets, "department": dept_targets},
    epochs=2,
    batch_size=32,
)

## A toy ResNet model

In [ ]:
inputs = keras.Input(shape=(32, 32, 3), name="img")
x = layers.Conv2D(32, 3, activation="relu")(inputs)
x = layers.Conv2D(64, 3, activation="relu")(x)
block_1_output = layers.MaxPooling2D(3)(x)

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_1_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_2_output = layers.add([x, block_1_output])

x = layers.Conv2D(64, 3, activation="relu", padding="same")(block_2_output)
x = layers.Conv2D(64, 3, activation="relu", padding="same")(x)
block_3_output = layers.add([x, block_2_output])

x = layers.Conv2D(64, 3, activation="relu")(block_3_output)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation="relu")(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10)(x)

model = keras.Model(inputs, outputs, name="toy_resnet")
model.summary()

In [ ]:
keras.utils.plot_model(model, "mini_resnet.png", show_shapes=True)

In [ ]:
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

model.compile(
    optimizer=keras.optimizers.RMSprop(1e-3),
    loss=keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["acc"],
)
# We restrict the data to the first 1000 samples so as to limit execution time
# on Colab. Try to train on the entire dataset until convergence!
model.fit(x_train[:1000], y_train[:1000], batch_size=64, epochs=1, validation_split=0.2)

## Shared layers

In [ ]:
# Embedding for 1000 unique words mapped to 128-dimensional vectors
shared_embedding = layers.Embedding(1000, 128)

# Variable-length sequence of integers
text_input_a = keras.Input(shape=(None,), dtype="int32")

# Variable-length sequence of integers
text_input_b = keras.Input(shape=(None,), dtype="int32")

# Reuse the same layer to encode both inputs
encoded_input_a = shared_embedding(text_input_a)
encoded_input_b = shared_embedding(text_input_b)

## Extract and reuse nodes in the graph of layers

In [ ]:
vgg19 = keras.applications.VGG19()

In [ ]:
features_list = [layer.output for layer in vgg19.layers]

In [ ]:
feat_extraction_model = keras.Model(inputs=vgg19.input, outputs=features_list)

img = np.random.random((1, 224, 224, 3)).astype("float32")
extracted_features = feat_extraction_model(img)

## Extend the API using custom layers

In [ ]:
import tensorflow as tf
class CustomDense(layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)

In [ ]:
class CustomDense(layers.Layer):
    def __init__(self, units=32):
        super(CustomDense, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.w = self.add_weight(
            shape=(input_shape[-1], self.units),
            initializer="random_normal",
            trainable=True,
        )
        self.b = self.add_weight(
            shape=(self.units,), initializer="random_normal", trainable=True
        )

    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

    def get_config(self):
        return {"units": self.units}


inputs = keras.Input((4,))
outputs = CustomDense(10)(inputs)

model = keras.Model(inputs, outputs)
config = model.get_config()

new_model = keras.Model.from_config(config, custom_objects={"CustomDense": CustomDense})

## Mix-and-match API styles

In [ ]:
units = 32
timesteps = 10
input_dim = 5

# Define a Functional model
inputs = keras.Input((None, units))
x = layers.GlobalAveragePooling1D()(inputs)
outputs = layers.Dense(1)(x)
model = keras.Model(inputs, outputs)


class CustomRNN(layers.Layer):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.units = units
        self.projection_1 = layers.Dense(units=units, activation="tanh")
        self.projection_2 = layers.Dense(units=units, activation="tanh")
        # Our previously-defined Functional model
        self.classifier = model

    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.units))
        for t in range(inputs.shape[1]):
            x = inputs[:, t, :]
            h = self.projection_1(x)
            y = h + self.projection_2(state)
            state = y
            outputs.append(y)
        features = tf.stack(outputs, axis=1)
        print(features.shape)
        return self.classifier(features)


rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, timesteps, input_dim)))

In [ ]:
units = 32
timesteps = 10
input_dim = 5
batch_size = 16


class CustomRNN(layers.Layer):
    def __init__(self):
        super(CustomRNN, self).__init__()
        self.units = units
        self.projection_1 = layers.Dense(units=units, activation="tanh")
        self.projection_2 = layers.Dense(units=units, activation="tanh")
        self.classifier = layers.Dense(1)

    def call(self, inputs):
        outputs = []
        state = tf.zeros(shape=(inputs.shape[0], self.units))
        for t in range(inputs.shape[1]):
            x = inputs[:, t, :]
            h = self.projection_1(x)
            y = h + self.projection_2(state)
            state = y
            outputs.append(y)
        features = tf.stack(outputs, axis=1)
        return self.classifier(features)


# Note that you specify a static batch size for the inputs with the `batch_shape`
# arg, because the inner computation of `CustomRNN` requires a static batch size
# (when you create the `state` zeros tensor).
inputs = keras.Input(batch_shape=(batch_size, timesteps, input_dim))
x = layers.Conv1D(32, 3)(inputs)
outputs = CustomRNN()(x)

model = keras.Model(inputs, outputs)

rnn_model = CustomRNN()
_ = rnn_model(tf.zeros((1, 10, 5)))

## End to end example

In [ ]:
!git clone https://github.com/mlnuggets/tensorflow.git

In [ ]:
!mv tensorflow/data/labels-en.json labels-en.json

In [ ]:
!mv tensorflow/data/dataloader.py dataloader.py

In [ ]:
!python dataloader.py -j labels-en.json

In [ ]:
import pandas as pd
df = pd.read_json("labels-en.json")
df.head()

In [ ]:
def image_names(externalId):
    return f"{externalId}.png"

In [ ]:
df["image_path"] = df["externalId"].map(image_names)

In [ ]:
df.tail()

In [ ]:
age = []
hair = []
beard = []
mustache = []
eye = []
def get_answers(tasks):
    for all_tasks in tasks:
        all_it = all_tasks[0]
        for item in all_it['classifications']:
                
            if item['title'] == 'Age':
                age.append(item['answer'])
                
            if item['title'] == 'Hair Color':
                hair.append(item['answer'])
                
            if item['title'] == 'Beard Color':
                beard.append(item['answer'])
                
            if item['title'] == 'Mustache Color':
                mustache.append(item['answer'])
                
            if item['title'] == 'Eye Color':
                eye.append(item['answer'])
            

In [ ]:
get_answers(df['tasks'])

In [ ]:
df['age'] = age

In [ ]:
df['hair_color'] = hair

In [ ]:
df['beard_color'] = beard

In [ ]:
df['mustache_color'] = mustache

In [ ]:
df['eye_color'] = eye

In [ ]:
df['age'].nunique()

In [ ]:
df.head()

In [ ]:
from sklearn.preprocessing import LabelEncoder
gender_labelencoder = LabelEncoder()
age_labelencoder = LabelEncoder()
hair_labelencoder = LabelEncoder()
beard_labelencoder = LabelEncoder()
mustache_labelencoder = LabelEncoder()
glases_labelencoder = LabelEncoder()
eye_labelencoder = LabelEncoder()

df = df.assign(age = age_labelencoder.fit_transform(df["age"]))
df = df.assign(hair_color = hair_labelencoder.fit_transform(df["hair_color"]))
df = df.assign(beard_color = beard_labelencoder.fit_transform(df["beard_color"]))
df = df.assign(mustache_color = mustache_labelencoder.fit_transform(df["mustache_color"]))
df = df.assign(age = age_labelencoder.fit_transform(df["eye_color"]))
df = df.assign(eye_color = eye_labelencoder.fit_transform(df["eye_color"]))


In [ ]:
df.head()

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255, 
                                   shear_range=0.2,
                                   zoom_range=0.2, 
                                   horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.1,
                                   validation_split=0.2
                                   )
validation_gen = ImageDataGenerator(rescale=1./255,validation_split=0.2)

In [ ]:
image_size = (128, 128)
batch_size = 32
base_dir = 'assets'
target_columns = ['age',
                  'hair_color','beard_color','mustache_color','eye_color',
                 ]

In [ ]:
type(target_columns)

In [ ]:

training_set = train_datagen.flow_from_dataframe(df,base_dir,
                                                seed=101,                                                 
                                                target_size=image_size,
                                                batch_size=batch_size,
                                                x_col='image_path',
                                                y_col=target_columns,
                                                subset = 'training',
                                                class_mode='multi_output')
validation_set = validation_gen.flow_from_dataframe(df,base_dir, 
                                              target_size=image_size,
                                              batch_size=batch_size, 
                                              x_col='image_path',
                                              y_col=target_columns,
                                              subset = 'validation',
                                              class_mode='multi_output'
                                                   )

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in training_set:
    for i in range(25):
        ax = plt.subplot(5, 5, i + 1)
        plt.imshow(images[i])
        plt.axis("off")
    break

In [ ]:
# training_set.labels

In [ ]:
# for x,y in training_set:
#     print(x, y)
#     break

In [ ]:
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPooling2D,Flatten,Dropout,Resizing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping
import numpy as np


In [ ]:
image_input = keras.Input(shape=(image_size[0], image_size[0], 3), name="images_input") 

x = Conv2D(filters=32,kernel_size=(3,3),activation="relu",name="first_block_conv2d")(image_input)
x = MaxPooling2D(pool_size=(2,2),name="first_block_maxpool2d")(x)
first_block_output = Flatten(name="first_block_flatten")(x)

x = Conv2D(filters=32,kernel_size=(3,3), activation='relu', name="second_block_conv2d")(image_input)
x = MaxPooling2D(pool_size=(2,2),name="second_block_maxpool2d")(x)
x = Flatten(name="second_block_flatten")(x)
second_block_output = layers.add([x, first_block_output], name="second_block_add")

x = Conv2D(filters=32,kernel_size=(3,3), activation='relu',name="third_block_conv2d")(image_input)
x = MaxPooling2D(pool_size=(2,2),name="third_block_maxpool2d")(x)
x = Flatten(name="third_block_flatten")(x)
third_block_output = layers.add([x, second_block_output],name="third_block_add")


x = Dropout(0.25, name="dropout1")(third_block_output)
x = Dense(128, activation="relu", name="dense1")(x)

age_prediction = Dense(df["age"].nunique(), activation="softmax",name="dense_age")(x)
hair_prediction = Dense(df["hair_color"].nunique(), activation="softmax",name="dense_hair")(x)
beard_prediction = Dense(df["beard_color"].nunique(), activation="softmax",name="dense_beard")(x)
mustache_prediction = Dense(df["mustache_color"].nunique(), activation="softmax",name="dense_mustache")(x)
eye_prediction = Dense(df["eye_color"].nunique(), activation="softmax",name="dense_eye")(x)

model = keras.Model(
    inputs=image_input,
    outputs=[
             age_prediction,
             hair_prediction,
        beard_prediction,
             mustache_prediction,
        eye_prediction,
    ],
)
keras.utils.plot_model(model, "model.png",show_shapes=True)

In [ ]:
model.compile(keras.optimizers.RMSprop(), loss='sparse_categorical_crossentropy', metrics='sparse_categorical_accuracy')

In [ ]:
callback = EarlyStopping(monitor='loss', patience=3)

In [ ]:
epochs=100
history = model.fit(training_set,validation_data=validation_set, epochs=epochs, callbacks = [callback])

In [ ]:
metrics_df = pd.DataFrame(history.history)

In [ ]:
metrics_df.head()

In [ ]:
model.evaluate(validation_set)

In [ ]:
metrics_df[["loss","val_loss"]].plot();

In [ ]:
metrics_df[["dense_age_loss","val_dense_age_loss"]].plot()

In [ ]:
metrics_df[["dense_beard_loss","val_dense_beard_loss"]].plot()

In [ ]:
metrics_df[["dense_hair_loss","val_dense_hair_loss"]].plot()

In [ ]:
metrics_df[["dense_beard_loss","val_dense_beard_loss"]].plot()

In [ ]:
metrics_df[["dense_mustache_loss","val_dense_mustache_loss"]].plot()

In [ ]:
metrics_df[["dense_eye_loss","val_dense_eye_loss"]].plot()

In [ ]:
# metrics_df[["dense_glasses_loss","val_dense_glasses_loss"]].plot()

In [ ]:
metrics_df.columns

In [ ]:
metrics_df[["dense_eye_sparse_categorical_accuracy","val_dense_eye_sparse_categorical_accuracy"]].plot()

In [ ]:
metrics_df[["dense_eye_sparse_categorical_accuracy","val_dense_eye_sparse_categorical_accuracy"]].plot()

In [ ]:
metrics_df[["dense_mustache_sparse_categorical_accuracy","val_dense_mustache_sparse_categorical_accuracy"]].plot()

In [ ]:
metrics_df[["dense_beard_sparse_categorical_accuracy","val_dense_beard_sparse_categorical_accuracy"]].plot()

In [ ]:
metrics_df[["dense_hair_sparse_categorical_accuracy","val_dense_hair_sparse_categorical_accuracy"]].plot()

In [ ]:
metrics_df[["dense_age_sparse_categorical_accuracy","val_dense_age_sparse_categorical_accuracy"]].plot()

## Make predictions

In [ ]:
image_url = "https://storage.googleapis.com/ango-covid-dataset/ffhq-dataset/batch2/25000.png"
image_path = keras.utils.get_file('Sample_Food', origin=image_url)
test_image = keras.utils.load_img(
    image_path, target_size=(image_size[0], image_size[1])
)
import matplotlib.pyplot as plt
plt.axis("off")
plt.imshow(test_image);

In [ ]:
img_array = tf.keras.utils.img_to_array(test_image)
img_array = tf.expand_dims(img_array, 0)

In [ ]:
img_array = img_array / 255.0
predictions = model.predict(img_array)
predictions

In [ ]:
age_predictions = predictions[0]
hair_predictions = predictions[1]
beard_predictions = predictions[2]
mustache_predictions = predictions[3]
eye_predictions = predictions[4]
age_predictions

In [ ]:
age_labelencoder.classes_

In [ ]:
def make_face_prediction(image_url):
    import tensorflow as tf
    import numpy as np
    image_path = keras.utils.get_file('Face', origin=image_url)
    test_image = keras.utils.load_img(
    image_path, target_size=(image_size[0], image_size[1]))
    img_array = tf.keras.utils.img_to_array(test_image)
    img_array = tf.expand_dims(img_array, 0) 
    img_array = img_array / 255.0
    predictions = model.predict(img_array)
    age_predictions = predictions[0][0]
    hair_predictions = predictions[1][0]
    beard_predictions = predictions[2][0]
    mustache_predictions = predictions[3][0]
    eye_predictions = predictions[4][0]
        
    age_scores = tf.nn.softmax(age_predictions).numpy()
    
    hair_scores = tf.nn.softmax(hair_predictions).numpy()
    
    beard_scores = tf.nn.softmax(beard_predictions).numpy()
    
    mustache_scores = tf.nn.softmax(mustache_predictions).numpy()
    
    eye_scores = tf.nn.softmax(eye_predictions).numpy()
    
    
    print(f"Age: {list(age_labelencoder.classes_)[np.argmax(age_scores)]} with a { (100 * np.max(age_scores)).round(2) } percent confidence.")
    print(f"Hair Color: {list(hair_labelencoder.classes_)[np.argmax(hair_scores)]} with a { (100 * np.max(hair_scores)).round(2) } percent confidence.") 
    print(f"Beard Color: {list(beard_labelencoder.classes_)[np.argmax(beard_scores)]} with a { (100 * np.max(beard_scores)).round(2) } percent confidence.") 
    print(f"Mustache Color: {list(mustache_labelencoder.classes_)[np.argmax(mustache_scores)]} with a { (100 * np.max(mustache_scores)).round(2) } percent confidence.") 
    print(f"Eye Color: {list(eye_labelencoder.classes_)[np.argmax(eye_scores)]} with a { (100 * np.max(eye_scores)).round(2) } percent confidence.") 
    

In [ ]:
make_face_prediction('https://storage.googleapis.com/ango-covid-dataset/ffhq-dataset/batch2/25000.png')

## Appendix: How to add layers

In [ ]:
input_shape = (2, 3, 4)
x1 = tf.random.normal(input_shape)
x2 = tf.random.normal(input_shape)
y = tf.keras.layers.Add()([x1, x2])
print(y.shape)

In [ ]:
x1

In [ ]:
x2

In [ ]:
y

In [ ]:
1.1024796 +  0.02846527

## Where to go from here
Follow us on [LinkedIn](https://www.linkedin.com/company/mlnuggets), [Twitter](https://twitter.com/ml_nuggets), [GitHub](https://github.com/mlnuggets) and subscribe to our [blog](https://www.machinelearningnuggets.com/#/portal) so that you don't miss a new issue.